In [1]:
import os

os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())

Current Working Directory  D:\02_batch_2


#### Directory Structure

In [4]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



In [6]:
list_files(r"D:\02_batch_2")

02_batch_2/
    37_degrees_series/
        37_degrees_series_0-0.txt
        37_degrees_series_0-0.wav
        37_degrees_series_0-1.txt
        37_degrees_series_0-1.wav
        37_degrees_series_0-2.txt
        37_degrees_series_0-2.wav
        37_degrees_series_0-3.txt
        37_degrees_series_0-3.wav
        37_degrees_series_0-4.txt
        37_degrees_series_0-4.wav
        37_degrees_series_0-5.txt
        37_degrees_series_0-5.wav
        37_degrees_series_0-6.txt
        37_degrees_series_0-6.wav
        37_degrees_series_1-0.txt
        37_degrees_series_1-0.wav
        37_degrees_series_1-1.txt
        37_degrees_series_1-1.wav
        37_degrees_series_1-2.txt
        37_degrees_series_1-2.wav
        37_degrees_series_1-3.txt
        37_degrees_series_1-3.wav
        37_degrees_series_1-4.txt
        37_degrees_series_1-4.wav
        37_degrees_series_1-5.txt
        37_degrees_series_1-5.wav
        37_degrees_series_1-6.txt
        37_degrees_series_1-6.wav
        37_de